**To Do**

- [ ] Ensure missing samples are accounted for

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from scipy.stats import mannwhitneyu, shapiro

# from bokeh.plotting import figure, show
# from bokeh.models import ColumnDataSource, CustomJS, Button
# from bokeh.io import output_notebook
# from bokeh.transform import factor_cmap, linear_cmap
# from bokeh.palettes import Category20
# from bokeh.layouts import column
# output_notebook()
from itertools import repeat, cycle

# %load_ext rpy2.ipython

In [2]:
# %%R
# library(ggplot2)

In [3]:
date_of_processing = "2023-02-10"

# Loading & Configuration

In [4]:
# experiment_path = Path("./results/aa5b6af/")
# results = pd.read_csv(experiment_path / "archive.csv.gz")
# results_gomea_shuf = pd.read_csv("./results/2023-01-20-gomea-shuffled-nkl-1c8e2cb-archive.csv.gz")
# results_gomea_shuf["algorithm_type"] = results_gomea_shuf["algorithm_type"] + "-shuf"

# results_imm_gomea_shuf = pd.read_csv("./results/2023-01-19-imm-gomea-shuffled-nkl-6520a07-archive.csv.gz")
# results_imm_gomea_shuf["algorithm_type"] = results_imm_gomea_shuf["algorithm_type"] + "-shuf"

results = pd.concat([
    # - still re-running
    pd.read_csv("./results/2022-12-13-scalability-gomea-nkl-re-149d6ac-archive.csv.gz"),

    pd.read_csv("./results/2022-12-13-scalability-ecga-nkl-re-149d6ac-archive.csv.gz"),
    pd.read_csv("./results/2022-12-13-scalability-ga-nkl-re-149d6ac-archive.csv.gz"),
    # results_gomea_shuf,
    # gomea - imm
    # results_imm_gomea_shuf,
    pd.read_csv("./results/2023-01-11-imm-gomea-nkl-f418675-archive.csv.gz"),
    # pd.read_csv("./results/2023-01-27-ss-gomea-nkl-d4d7bed-archive.csv.gz"),
])

remapping_algorithm_type = {
    "sync": "ecga-sync",
    "async-throttled": "ecga-async-th",
}
results["algorithm_type"] = results["algorithm_type"].replace(remapping_algorithm_type)

problem_columns = ["problem", "l", "runtime_type"]
approach_columns = ["replacement_strategy", "algorithm_type", "tournament_size"]
run_columns = ["seed"]

In [5]:
def relabel(df):
    # Relabeling
    # - Approach
    df["approach"] = "GA"
    df.loc[df["algorithm_type"].str.contains("ecga"), "approach"] = "ECGA"
    df.loc[df["algorithm_type"].str.contains("gomea"), "approach"] = "GOMEA"
    df.loc[(df["algorithm_type"].str.contains("gomea") & df["algorithm_type"].str.contains("immidiate")), "approach"] = "GOMEA-IMM"
    # df.loc[(df["algorithm_type"].str.contains("gomea") & df["algorithm_type"].str.contains("steady-state")), "approach"] = "SS-GOMEA"
    # df.loc[df["algorithm_type"].str.contains("shuf"), "approach"] = df.loc[df["algorithm_type"].str.contains("shuf"), "approach"] + "-SHUF"
    # df["approach"] = pd.Categorical(df["approach"], categories=["GOMEA", "GOMEA-SHUF", "GOMEA-IMM", "GOMEA-IMM-SHUF", "ECGA", "GA"], ordered=True)
    df["approach"] = pd.Categorical(df["approach"], categories=["GOMEA", "GOMEA-IMM", "ECGA", "GA"], ordered=True) #, "SS-GOMEA"

    # - selection
    df["selection"] = "-"
    df.loc[df["replacement_strategy"] == 5, "selection"] = "steady-state"
    df.loc[df["replacement_strategy"] == 6, "selection"] = "generational"
    df["selection"] = pd.Categorical(df["selection"], categories=["-", "steady-state", "generational"], ordered=True)
    # - CX
    df["cx"] = "-"
    df.loc[df["algorithm_type"].str.contains("subfunction"), "cx"] = "SFX"
    df.loc[df["algorithm_type"].str.contains("uniform"), "cx"] = "UX"
    df.loc[df["algorithm_type"].str.contains("twopoint"), "cx"] = "TPX"
    df["cx"] = pd.Categorical(df["cx"], categories=["-", "UX", "TPX", "SFX"], ordered=True)
    # - (a)sync 
    df["(a)sync"] = "s"
    df.loc[df["algorithm_type"].str.contains("async"), "(a)sync"] = "a"
    df["(a)sync"] = pd.Categorical(df["(a)sync"], categories=["a", "s"], ordered=True)
    # - timing distribution
    # df["time (v)"] = df["runtime_type"].replace({
    #     "cheap-optimum-100": 100/1,
    #     "cheap-optimum-10":10/1,
    #     "cheap-optimum": 2/1,
    #     "constant": 1/1,
    #     "expensive-optimum": 1/2,
    #     "expensive-optimum-10": 1/10,
    #     "expensive-optimum-100": 1/100,
    # })
    df["timing"] = pd.Categorical(df["runtime_type"].replace({
        "cheap-optimum-100": "100:1",
        "cheap-optimum-10": "10:1",
        "cheap-optimum": "2:1",
        "constant": "1:1",
        "expensive-optimum": "1:2",
        "expensive-optimum-10": "1:10",
        "expensive-optimum-100": "1:100",
    }), categories = ["100:1", "10:1", "2:1", "1:1", "1:2", "1:10", "1:100"], ordered=True)

In [6]:
results[~np.isnan(results["population_size"])]

,Unnamed: 0,#evaluations,simulation time (s),objectives,archive ordinal,archive ordinals removed,genotype (categorical),problem,seed,l,vtr,instance,replacement_strategy,tournament_size,runtime_type,algorithm_type,population_size,num_procs
0,0,40.0,1.0,-8.5354,1.0,NaN,1 1 0 0 1 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 ...,nk-landscape,125,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-async,40.0,40.0
1,1,40.0,1.0,-8.5398,2.0,1.0,1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 ...,nk-landscape,125,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-async,40.0,40.0
2,2,40.0,1.0,-10.1881,3.0,2.0,0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 0 1 1 0 0 0 1 ...,nk-landscape,125,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-async,40.0,40.0
3,3,40.0,1.0,-11.1385,4.0,3.0,0 0 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 1 0 1 0 0 1 ...,nk-landscape,125,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-async,40.0,40.0
4,4,40.0,1.0,-12.5149,5.0,4.0,1 1 0 1 1 0 1 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1 0 ...,nk-landscape,125,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-async,40.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36480,13,6324.0,93.0,-15.4074,14.0,13.0,0 1 0 0 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 1 0 1 0 ...,nk-landscape,59,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-immidiate-sync,68.0,68.0
36481,14,12237.0,179.0,-15.4223,15.0,14.0,0 1 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,nk-landscape,59,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-immidiate-sync,68.0,68.0
36482,15,12373.0,181.0,-15.6187,16.0,15.0,0 1 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,nk-landscape,59,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-immidiate-sync,68.0,68.0
36483,16,14137.0,207.0,-15.7418,17.0,16.0,1 1 1 1 1 0 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,nk-landscape,59,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-immidiate-sync,68.0,68.0


In [7]:
results[np.isnan(results["population_size"])]

,Unnamed: 0,#evaluations,simulation time (s),objectives,archive ordinal,archive ordinals removed,genotype (categorical),problem,seed,l,vtr,instance,replacement_strategy,tournament_size,runtime_type,algorithm_type,population_size,num_procs
864,0,NaN,NaN,NaN,NaN,NaN,NaN,nk-landscape,73,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,5,4,expensive-optimum-10,ga-uniform-async,NaN,NaN
924,0,NaN,NaN,NaN,NaN,NaN,NaN,nk-landscape,85,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,5,4,constant,ga-uniform-async,NaN,NaN
965,0,NaN,NaN,NaN,NaN,NaN,NaN,nk-landscape,68,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,5,4,expensive-optimum-100,ga-uniform-async,NaN,NaN
1080,0,NaN,NaN,NaN,NaN,NaN,NaN,nk-landscape,56,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,5,4,expensive-optimum-10,ga-uniform-async,NaN,NaN
1104,0,NaN,NaN,NaN,NaN,NaN,NaN,nk-landscape,103,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,5,4,expensive-optimum-100,ga-uniform-async,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180935,0,NaN,NaN,NaN,NaN,NaN,NaN,nk-landscape,71,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,5,4,expensive-optimum-100,ga-uniform-async,NaN,NaN
180963,0,NaN,NaN,NaN,NaN,NaN,NaN,nk-landscape,116,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,5,4,constant,ga-uniform-async,NaN,NaN
181092,0,NaN,NaN,NaN,NaN,NaN,NaN,nk-landscape,63,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,5,4,constant,ga-uniform-async,NaN,NaN
181792,0,NaN,NaN,NaN,NaN,NaN,NaN,nk-landscape,62,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,5,4,expensive-optimum-10,ga-subfunction-async,NaN,NaN


In [8]:
results

,Unnamed: 0,#evaluations,simulation time (s),objectives,archive ordinal,archive ordinals removed,genotype (categorical),problem,seed,l,vtr,instance,replacement_strategy,tournament_size,runtime_type,algorithm_type,population_size,num_procs
0,0,40.0,1.0,-8.5354,1.0,NaN,1 1 0 0 1 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 ...,nk-landscape,125,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-async,40.0,40.0
1,1,40.0,1.0,-8.5398,2.0,1.0,1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 ...,nk-landscape,125,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-async,40.0,40.0
2,2,40.0,1.0,-10.1881,3.0,2.0,0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 0 1 1 0 0 0 1 ...,nk-landscape,125,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-async,40.0,40.0
3,3,40.0,1.0,-11.1385,4.0,3.0,0 0 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 1 0 1 0 0 1 ...,nk-landscape,125,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-async,40.0,40.0
4,4,40.0,1.0,-12.5149,5.0,4.0,1 1 0 1 1 0 1 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1 0 ...,nk-landscape,125,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-async,40.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36480,13,6324.0,93.0,-15.4074,14.0,13.0,0 1 0 0 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 1 0 1 0 ...,nk-landscape,59,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-immidiate-sync,68.0,68.0
36481,14,12237.0,179.0,-15.4223,15.0,14.0,0 1 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,nk-landscape,59,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-immidiate-sync,68.0,68.0
36482,15,12373.0,181.0,-15.6187,16.0,15.0,0 1 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,nk-landscape,59,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-immidiate-sync,68.0,68.0
36483,16,14137.0,207.0,-15.7418,17.0,16.0,1 1 1 1 1 0 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,nk-landscape,59,40,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,0,4,constant,gomea-immidiate-sync,68.0,68.0


In [9]:
# Compute the hitting evaluation
# def select_hit(df):
#     return df.iloc[df["archive ordinal"].argmax()]
# results_hit = results.groupby(problem_columns + approach_columns + run_columns).apply(select_hit).reset_index(drop=True).fillna(np.inf)
results_hit = results.sort_values("archive ordinal").groupby(problem_columns + approach_columns + run_columns).last().reset_index().fillna(np.inf)

In [10]:
results_hit

,problem,l,runtime_type,replacement_strategy,algorithm_type,tournament_size,seed,Unnamed: 0,#evaluations,simulation time (s),objectives,archive ordinal,archive ordinals removed,genotype (categorical),vtr,instance,population_size,num_procs
0,nk-landscape,40,cheap-optimum,0,gomea-async,4,42,24,25627.0,893.525,-15.7942,25.0,24.0,1 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,40.0,40.0
1,nk-landscape,40,cheap-optimum,0,gomea-async,4,43,22,28090.0,651.475,-15.7942,23.0,22.0,1 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,60.0,60.0
2,nk-landscape,40,cheap-optimum,0,gomea-async,4,44,30,15662.0,675.975,-15.7942,31.0,30.0,1 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,32.0,inf
3,nk-landscape,40,cheap-optimum,0,gomea-async,4,45,23,20679.0,901.400,-15.7942,24.0,23.0,1 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,32.0,inf
4,nk-landscape,40,cheap-optimum,0,gomea-async,4,46,27,31510.0,618.225,-15.7942,28.0,27.0,1 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,72.0,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,nk-landscape,40,expensive-optimum-100,6,ga-uniform-sync,4,137,24,31744.0,1374.775,-15.7942,25.0,24.0,1 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,1984.0,1984.0
13996,nk-landscape,40,expensive-optimum-100,6,ga-uniform-sync,4,138,21,91724.0,2052.500,-15.7942,22.0,21.0,1 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,3988.0,3988.0
13997,nk-landscape,40,expensive-optimum-100,6,ga-uniform-sync,4,139,26,378112.0,19694.200,-15.7942,27.0,26.0,1 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,1792.0,1792.0
13998,nk-landscape,40,expensive-optimum-100,6,ga-uniform-sync,4,140,41,494704.0,2493.100,-15.7942,42.0,41.0,1 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 ...,15.7942,../EALib/instances/nk/instances/n5_s2/L40/1.txt,17668.0,17668.0


# Required population size

In [45]:
results_hit_f = results_hit.copy()# [(results_hit["replacement_strategy"] == 5) | (results_hit["replacement_strategy"] == 6)]
results_hit_f["population_size"] = np.where(np.isfinite(results_hit_f["population_size"]), results_hit_f["population_size"], np.inf)
approach_columns_r = approach_columns#["algorithm_type", "replacement_strategy"]

population_size_required = results_hit_f.groupby(problem_columns + approach_columns_r)["population_size"].median()
table = population_size_required.reset_index().sort_values(approach_columns_r).pivot(index=problem_columns, columns=approach_columns_r).style.format(precision=0)
# table

In [46]:
pt = population_size_required.rename("minimally required population size").reset_index().copy()
relabel(pt)
    
pt = pt\
    .pivot_table(index=["timing"], columns=["selection", "approach", "cx", "(a)sync"], values="minimally required population size", sort=False)\
    .sort_index()\
    .replace({np.inf: np.nan})

tab = pt.style.format(precision=0, na_rep="-")
tab.to_excel(f"{date_of_processing}-nklandscape-popsize.xlsx")
tab


In [47]:
results_hit_f = results_hit.copy()# [(results_hit["replacement_strategy"] == 5) | (results_hit["replacement_strategy"] == 6)]
results_hit_f["population_size"] = np.where(np.isfinite(results_hit_f["population_size"]), results_hit_f["population_size"], np.inf)
approach_columns_r = approach_columns#["algorithm_type", "replacement_strategy"]
population_size_required = results_hit_f.groupby(problem_columns + approach_columns_r)["population_size"].quantile([0.25, 0.50, 0.75])
population_size_required
# table = population_size_required.reset_index().sort_values(approach_columns_r).pivot(index=problem_columns, columns=approach_columns_r).style.format(precision=0)
pt = population_size_required.rename("minimally required population size").reset_index().rename(columns={"level_6": "quantile"}).copy()
pt["quantile"] = [f"{q:.2f}" for q in pt["quantile"]]
relabel(pt)
# Drop GOMEA-IMM sync - it is identical in behavior.
pt = pt[(pt["approach"] != "GOMEA-IMM") | (pt["(a)sync"] != "s")]
str_c = pt["(a)sync"].astype(str)
str_c[(pt["approach"] == "GOMEA-IMM") & (pt["(a)sync"] == "a")] = "a/i"
str_c[(pt["approach"] == "GOMEA") & (pt["(a)sync"] == "a")] = "a/e"
pt["(a)sync"] = pd.Categorical(str_c, ["a", "a/e", "a/i", "s"])
pt["approach"] = pt["approach"].replace({"GOMEA-IMM": "GOMEA"})

pt = pt\
    .pivot_table(index=["timing"], columns=["selection", "approach", "cx", "(a)sync", "quantile"], values="minimally required population size", sort=False)\
    .sort_index()\
    .replace({np.inf: np.nan})

tab = pt.style.format(precision=0, na_rep="-")
tab.to_excel(f"{date_of_processing}-nklandscape-popsize-extended.xlsx")
tab

In [48]:
# 
ps = results_hit_f.copy()
relabel(ps)
# Drop GOMEA-IMM sync - it is identical in behavior.
ps = ps[(ps["approach"] != "GOMEA-IMM") | (ps["(a)sync"] != "s")]
str_c = ps["(a)sync"].astype(str)
str_c[(ps["approach"] == "GOMEA-IMM") & (ps["(a)sync"] == "a")] = "a/i"
str_c[(ps["approach"] == "GOMEA") & (ps["(a)sync"] == "a")] = "a/e"
ps["(a)sync"] = pd.Categorical(str_c, ["a", "a/e", "a/i", "s"], ordered=True)
ps["approach"] = ps["approach"].replace({"GOMEA-IMM": "GOMEA"})

#
c = ["selection", "approach", "cx", "(a)sync"]
ranges = ps[ps["timing"] == "1:1"].groupby(c)["population_size"].agg([np.median, np.mean, np.std, np.nanmin, np.nanmax]).reset_index()
# ranges = ps.groupby(c)["population_size"].agg([np.nanmin, np.nanmax]).reset_index()
ps = pd.merge(ps, ranges, on=c)
# 1: Normalize to unit normal
# ps["normalized population size"] = (ps["population_size"] - ps["mean"]) / ps["std"]
# 2: Normalize to min-max...
# ps["normalized population size"] = ((ps["population_size"] - ps["nanmin"]) / (ps["nanmax"] - ps["nanmin"])).fillna(0.5)
# 3: Median based normalization - preserves scale differences within approach, but places
#    different approaches in the same order of magnitude
#    - alternative interpretation: how big of a factor difference is there?
ps["normalized population size"] = (ps["population_size"] - ps["median"]) / ps["median"]

columns_of_interest = ["problem", "timing", "population_size", "normalized population size", "approach", "cx", "(a)sync", "selection"]
ps = ps[columns_of_interest].sort_values("timing")
ps.to_csv(f"{date_of_processing}-summary-nkl.csv.gz")

In [25]:
# # Statistical Test against differently timed variants
# use_continuity = True
# problem_columns_not_runtime = [a for a in problem_columns if a != "runtime_type"]
# samples_per_config = results_hit_f.groupby(problem_columns + approach_columns)["population_size"].agg(list).reset_index()
# runtime_pairings = pd.merge(samples_per_config, samples_per_config, on=problem_columns_not_runtime + approach_columns)
# # runtime_pairings["statistical_test"] = runtime_pairings.apply(lambda r: mannwhitneyu(r["population_size_x"], r["population_size_y"]), axis=1)
# runtime_pairings["statistical_test"] = runtime_pairings.apply(lambda r: mannwhitneyu(r["population_size_x"], r["population_size_y"], use_continuity=use_continuity).pvalue, axis=1)
# columns_to_keep = problem_columns_not_runtime + approach_columns + ["runtime_type_x", "runtime_type_y", "statistical_test"]
# paired_test_result = runtime_pairings[columns_to_keep].pivot(index=problem_columns_not_runtime + approach_columns + ["runtime_type_x"], columns=["runtime_type_y"]).style
# paired_test_result.to_excel(f"{date_of_processing}-nklandscape-stattest.xlsx")

In [73]:
# Statistical Test against differently timed variants
p = 0.05
use_continuity = True
problem_columns_not_runtime = [a for a in problem_columns if a != "runtime_type"]
samples_per_config = results_hit_f.groupby(problem_columns + approach_columns)["population_size"].agg(list).reset_index()
relabel(samples_per_config)
samples_per_config = samples_per_config[(samples_per_config["approach"] != "GOMEA-IMM") | (samples_per_config["(a)sync"] != "s")]
str_c = samples_per_config["(a)sync"].astype(str)
str_c[(samples_per_config["approach"] == "GOMEA-IMM") & (samples_per_config["(a)sync"] == "a")] = "a/i"
str_c[(samples_per_config["approach"] == "GOMEA") & (samples_per_config["(a)sync"] == "a")] = "a/e"
samples_per_config["(a)sync"] = pd.Categorical(str_c, ["a", "a/e", "a/i", "s"], ordered=True)
samples_per_config["approach"] = samples_per_config["approach"].replace({"GOMEA-IMM": "GOMEA"})

samples_per_config["timing"] = pd.Categorical(samples_per_config["timing"], categories=["100:1", "10:1", "2:1", "1:1", "1:2", "1:10", "1:100"])
relabeled_approach_columns = ["selection", "approach", "cx", "(a)sync"]
runtime_pairings = pd.merge(samples_per_config, samples_per_config, on=problem_columns_not_runtime + relabeled_approach_columns)
# runtime_pairings
# runtime_pairings["statistical_test"] = runtime_pairings.apply(lambda r: mannwhitneyu(r["population_size_x"], r["population_size_y"]), axis=1)
runtime_pairings[["U", "p_value"]]  = runtime_pairings.apply(lambda r: pd.Series(mannwhitneyu(r["population_size_x"], r["population_size_y"], use_continuity=use_continuity)), axis=1)
columns_to_keep = problem_columns_not_runtime + relabeled_approach_columns + ["timing_x", "timing_y", "p_value", "U"]
paired_test_result = runtime_pairings[columns_to_keep].pivot_table(index=relabeled_approach_columns + ["timing_x"], columns=["timing_y"], values=["p_value"])
# paired_test_result.to_excel(f"{date_of_processing}-nklandscape-stattest.xlsx")
paired_test_result_f = paired_test_result.style.format("{:.3f}")

def mark_significance_row(x):
    # x is a row
    o = np.argsort(x)
    o_inv = np.argsort(o)
    # test against 1/i - though in this case one of the samples is
    # ourselves - exclude that from the count.
    rcs = np.concatenate([np.flip(np.arange(len(x) - 1) + 1), [1]])
    ps = p / rcs
    s = x[o] < ps
    s = np.cumsum(s) == np.arange(len(x)) + 1
    return np.where(s[o_inv], "color: red", "")

# paired_test_result_f = paired_test_result_f.applymap(lambda x: np.where(x < 0.05, "color: red", ""))
paired_test_result_f.apply(mark_significance_row, axis=1)
paired_test_result_f.to_excel(f"{date_of_processing}-nklandscape-stattest-p.xlsx")

paired_test_result_U = runtime_pairings[columns_to_keep].pivot_table(index=relabeled_approach_columns + ["timing_x"], columns=["timing_y"], values=["U"])
paired_test_result_U_f = paired_test_result_U.style.format("{:i}")
paired_test_result_U_f.to_excel(f"{date_of_processing}-nklandscape-stattest-U.xlsx")

paired_test_result_f

In [22]:
success_rate = (results_hit["population_size"] != np.inf).groupby([results_hit[x] for x in (problem_columns + approach_columns_r)]).mean()
table = success_rate.reset_index().sort_values(approach_columns_r).pivot(index=problem_columns, columns=approach_columns_r).style.format(precision=3)
table.to_excel(f"{date_of_processing}-nklandscape-success-rate.xlsx")
table

In [23]:
sample_count = results_hit["population_size"].groupby([results_hit[x] for x in (problem_columns + approach_columns_r)]).count()
table = sample_count.reset_index().sort_values(approach_columns_r).pivot(problem_columns, approach_columns_r).style.format(precision=3)
table.to_excel(f"{date_of_processing}-nklandscape-sample-count.xlsx")
table

/tmp/ipykernel_4700/1731172668.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  table = sample_count.reset_index().sort_values(approach_columns_r).pivot(problem_columns, approach_columns_r).style.format(precision=3)


# Evaluations required

In [10]:
evaluations_required = results_hit.groupby(problem_columns + approach_columns)["#evaluations"].median()
table = evaluations_required.reset_index().sort_values(approach_columns).pivot(problem_columns, approach_columns).style
table.to_excel(f"{date_of_processing}-nklandscape-evaluations-required-median.xlsx")
table

In [ ]:
evaluations_required = results_hit.groupby(problem_columns + approach_columns)["#evaluations"].quantile(0.95)
evaluations_required.reset_index().sort_values(approach_columns).pivot(problem_columns, approach_columns).style

In [ ]:
evaluations_required = results_hit.groupby(problem_columns + approach_columns)["#evaluations"].quantile([0.05, 0.5, 0.95])
evaluations_required = evaluations_required.unstack().reset_index().rename(columns={
    0.05: "0.05",
    0.50: "0.50",
    0.95: "0.95",
})
evaluations_required["approach"] = evaluations_required["algorithm_type"] + "_" + evaluations_required["replacement_strategy"].map(str) + "_" + evaluations_required["tournament_size"].map(str)
evaluations_required["problem and t"] = evaluations_required["problem"] + "_" + evaluations_required["runtime_type"]

In [ ]:
s = evaluations_required[(evaluations_required["problem"] == "onemax") & (evaluations_required["runtime_type"] == "constant")]

c = Category20[20]

TOOLTIPS = [
    ("approach", "@approach"),
    ("l", "@l"),
    ("#evaluations", "@{0.50}")
]
f = figure(
    tools=["hover"],
    plot_width=800, plot_height=800
)
f.hover.tooltips = TOOLTIPS
# approach_colormapped = factor_cmap(field_name="approach", palette=Category20[20], factors=s["approach"].unique())
for (name, df), color in zip(s.groupby("approach"), cycle(c)):
    cds = ColumnDataSource(df)
    r = f.line(
        source=cds,
        x="l",
        y="0.50",
        color=color,
        line_width=2,
        legend_label = name)
    f.varea(
        source=cds,
        x="l",
        y1="0.95",
        y2="0.05",
        color=color,
        alpha=0.5,
        muted_alpha=0.0,
        legend_label = name
    )

# f.legend.click_policy = "mute"
f.legend.click_policy = "hide"
f.legend.orientation = "vertical"
f.legend.location = "top_left"

b = Button(label='Clear selection')
b.js_on_click(CustomJS(args=dict(cds=cds), code="cds.selected.indices = [];"))
show(f)

# (Simulated) Time required

In [11]:
sim_time_required = results_hit.groupby(problem_columns + approach_columns)["simulation time (s)"].median()
table = sim_time_required.reset_index().sort_values(approach_columns).pivot(problem_columns, approach_columns).style
table.to_excel(f"{date_of_processing}-nklandscape-sim-time-required-median.xlsx")
table

In [ ]:
sim_time_required = results_hit.groupby(problem_columns + approach_columns)["simulation time (s)"].quantile([0.05, 0.5, 0.95])
sim_time_required = sim_time_required.unstack().reset_index().rename(columns={
    0.05: "0.05",
    0.50: "0.50",
    0.95: "0.95",
})
sim_time_required["approach"] = sim_time_required["algorithm_type"] + "_" + sim_time_required["replacement_strategy"].map(str) + "_" + sim_time_required["tournament_size"].map(str)
sim_time_required["problem and t"] = sim_time_required["problem"] + "_" + sim_time_required["runtime_type"]

In [ ]:
s = sim_time_required[(sim_time_required["problem"] == "onemax") & (sim_time_required["runtime_type"] == "constant")]

c = Category20[20]

TOOLTIPS = [
    ("approach", "@approach"),
    ("l", "@l"),
    ("simulation time (s)", "@{0.50}")
]
f = figure(
    tools=["hover"],
    plot_width=800, plot_height=800
)
f.hover.tooltips = TOOLTIPS
# approach_colormapped = factor_cmap(field_name="approach", palette=Category20[20], factors=s["approach"].unique())
for (name, df), color in zip(s.groupby("approach"), cycle(c)):
    cds = ColumnDataSource(df)
    r = f.line(
        source=cds,
        x="l",
        y="0.50",
        color=color,
        line_width=2,
        legend_label = name)
    f.varea(
        source=cds,
        x="l",
        y1="0.95",
        y2="0.05",
        color=color,
        alpha=0.05,
        muted_alpha=0.0,
        legend_label = name
    )

# f.legend.click_policy = "mute"
f.legend.click_policy = "hide"
f.legend.orientation = "vertical"
f.legend.location = "top_left"

show(f)

# (Simulated) CPU Time required
As the previous metric can be minimized by evaluating the entire search space in parallel.
This metric multiplies by the number of parallel processors (in this case: the population size).
Note that if an approach does not scale perfectly, this score will be worse than single threaded.

In [ ]:
results_hit["simulated cpu time (s)"] = results_hit["simulation time (s)"] * results_hit["population_size"]
sim_cpu_time_required = results_hit.groupby(problem_columns + approach_columns)["simulated cpu time (s)"].mean()
sim_cpu_time_required